此內容可以讓我們建立code-server可進行遠端開發。
另外提供novnc的使用，可以看到圖形化的輸出。

首先，需填入自己主機外網連線資訊，由容器開啟的主機IP及SSH連接埠。
https://github.com/DevinY/colab

In [ ]:
%env SSH_SERVER=
%env SSH_PORT=

套件安裝



套件選項一: 安裝VNCServer要使用novnc，安裝較花時間。

In [ ]:
!apt-get -y update && apt-get -y install xfce4 xfce4-goodies tightvncserver net-tools vim autocutsel openssh-server ssh-askpass

選項二: 只安裝open-ssh，例如只想用code-server時使用。

In [ ]:
!apt-get -y update && apt-get -y install net-tools vim openssh-server

建立Openss金鑰

選項一: 產生不需要密碼的OpenSSH公鑰



In [ ]:
!ssh-keygen -t rsa -q -f "$HOME/.ssh/id_rsa" -N ""

選項二: 金鑰需有密碼才能連回我們自己的主機(使用有密碼的金鑰保護自己的主機)

In [ ]:
!ssh-keygen

In [ ]:
!ssh-agent

依上方的結果填入環境變數

In [ ]:
%env SSH_AGENT_PID=843
%env SSH_AUTH_SOCK=/tmp/ssh-fY2Lx3oDPk3r/agent.842

In [ ]:
!ssh-add

公鑰新增，查看下方公鑰結果，並將此公鑰加入到我們的colab容器中

In [ ]:
cat ~/.ssh/id_rsa.pub

驗證SSH連線，能看到回傳資料，才代表了SSH連線是正常的哦。

In [ ]:
!ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} netstat -ntlp

將遠端主機容器內的金鑰，新增到自己colab的authorzied_keys中

In [ ]:
!ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} sh -c ssh-keygen -t rsa -q -f "$HOME/.ssh/id_rsa" -N ""
!scp -P ${SSH_PORT} colab@${SSH_SERVER}:~/.ssh/id_rsa.pub ~/.ssh/authorized_keys

一、關於VNC Server

In [ ]:
%env USER=root
%env DISPLAY=:1

In [ ]:
!sh -c "USER=root;export USER;vncserver"

停止VNC Server(要停止VNC Server執行用)


In [ ]:
!vncserver -kill :1

novnc copy past

In [ ]:
!autocutsel -fork

檢查是否啟動了，會看到port 5901

In [ ]:
!netstat -ntlp

建立5901通道到遠端主機，遠端主機的連接埠

In [ ]:
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR 127.0.0.1:5901:0.0.0.0:5901&

二、公開colab的openssh連接埠



In [ ]:
!mkdir -p /run/sshd
!/usr/sbin/sshd

In [ ]:
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR \*:2222:0.0.0.0:22&

完成連線建立，我們可以刪除angent，防止別人由VNC連回我們的colab容器

In [ ]:
!eval $(ssh-agent -k)

以下指令，用於重新設定vnc密碼

In [ ]:
!vncpasswd

三、安裝code-server

----


In [ ]:
!curl -fsSL https://code-server.dev/install.sh | sh

啟動code-server在port 8081

In [ ]:
!nohup code-server --bind-addr 0.0.0.0:8081&

建立code-server通道

In [ ]:
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR \*:8080:0.0.0.0:8081&
!cat /root/.config/code-server/config.yaml